<div class="head0">
    <div class="head0__name">
        TT-test
    </div>
    <div class="head0__note">
        We perform some tests.
    </div>
</div>

In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from time import perf_counter as tpc

from fpcross import ij

import tt

ij()

Start |  2:56PM MSK on Feb 11, 2020 | python 3.7.4    |
-------------------------------------------------------


In [8]:
class tensor(object):
    
    def __init__(self, X=None, e=1E-6):
        self.comp(X, e)
            
    def comp(self, X, e=1E-6):
        self.X = X
        self.e = e
        self.d = 0 if X is None else len(X.shape)
        self.G = []
        
        if self.X is None:
            return
        
        Z, d, n = X.copy(), len(X.shape), X.shape
        e*= np.linalg.norm(X) / np.sqrt(d - 1)

        for k in range(d-1):
            Z = Z.reshape(-1, np.prod(n[(k+1):]), order='F')

            U, S, V = np.linalg.svd(Z, full_matrices=False)
            r = np.argmax(S<=e) or S.shape[0]
            S = np.diag(S[:r])
            U = U[:, :r]
            V = S @ V[:r, :]

            self.G.append(U.reshape((-1, n[k], r), order='F'))
            Z = V

        self.G.append(Z.reshape((-1, n[-1], 1), order='F'))
        
    def round(self, e=1E-6):
        self.e = e
        
        for k in range(self.d-1, 0, -1):
            q, n, r = self.G[k].shape
            U = self.G[k].reshape((q, n * r), order='F').T
            Q, R = np.linalg.qr(U)
            self.G[k] = Q.T.reshape((q, n, r), order='F')
            self.G[k-1] = np.tensordot(self.G[k-1], R.T, 1)
                
        for k in range(1, self.d-1):
            q, n, r = self.G[k].shape
            Z = self.G[k].reshape((q * n, r), order='F')
            U, S, V = np.linalg.svd(Z, full_matrices=False)
            r = np.argmax(S<=self.e) or S.shape[0]
            S = np.diag(S[:r])
            U = U[:, :r]
            V = S @ V[:r, :]
                
            self.G[k] = U.reshape((q, n, r), order='F')
            self.G[k+1] = np.tensordot(V, self.G[k+1], 1)
            
    def full(self):
        Z = self.G[0][0, :, :]
        for G_ in self.G[1:-1]:
            Z = np.tensordot(Z, G_, 1)
        Z = np.tensordot(Z, self.G[-1][:, :, 0], 1)
        
        return Z
    
    def info(self):
        n = [G_.shape[1] for G_ in self.G]
        s = [G_.size for G_ in self.G]
        r = ", ".join([str(G_.shape[2]) for G_ in self.G[:-1]])
        c = np.prod(n) / np.sum(s)
        print(f'Tensor d = {len(self.G):3d} | c = {c:8.1e} | n = {n} | r = [{r}]')

        if self.X is not None:
            Z = self.full()
            e = np.linalg.norm(self.X - Z)/np.linalg.norm(self.X)
            print(f'Norm           : {np.linalg.norm(Z):.2f}')
            print(f'Error          : {e:8.2e}')

In [9]:
n = [4, 6, 7, 9, 20]

X = np.zeros(n)
for n1 in range(n[0]):
    for n2 in range(n[1]):
        for n3 in range(n[2]):
            for n4 in range(n[3]):
                for n5 in range(n[4]):
                    X[n1, n2, n3, n4, n5] = 1 / (2 + n1 + n2**2 + np.sin(n3) + n3**3 + np.cos(n4**2))

Y = np.zeros(n)
for n1 in range(n[0]):
    for n2 in range(n[1]):
        for n3 in range(n[2]):
            for n4 in range(n[3]):
                for n5 in range(n[4]):
                    Y[n1, n2, n3, n4, n5] = 1 / (2 + n1**3 + n2 + np.cos(n3) + n3**2 + np.sin(n4**2))
                    
print(f'Norm X = {np.linalg.norm(X):.2f}')
print(f'Norm Y = {np.linalg.norm(Y):.2f}')

Norm X = 16.21
Norm Y = 15.87


In [10]:
print('---------------- Construction')

t = tpc()
Z = tensor(X, 1.E-6)
t = tpc() - t
Z.info()

print(f'Time (sec)     : {t:-8.4f}')

print('---------------- Rounding')

t = tpc()
Z.round(1E-2)
t = tpc() - t
Z.info()

print(f'Time (sec)     : {t:-8.4f}')

---------------- Construction
Tensor d =   5 | c =  5.5e+01 | n = [4, 6, 7, 9, 20] | r = [4, 8, 5, 1]
Norm           : 16.21
Error          : 6.24e-07
Time (sec)     :   0.0078
---------------- Rounding
Tensor d =   5 | c =  7.9e+01 | n = [4, 6, 7, 9, 20] | r = [4, 6, 4, 1]
Norm           : 16.21
Error          : 7.20e-04
Time (sec)     :   0.0019


In [11]:
print('---------------- Construction (ttpy)')

t = tpc()
Z = tt.tensor(X, 1E-6)
t = tpc() - t
e = np.linalg.norm(X - Z.full())/np.linalg.norm(X)

print(f'Ranks          : {Z.r[1:-1]}')
print(f'Error          : {e:8.2e}')
print(f'Time (sec)     : {t:-8.4f}')

print('---------------- Rounding (ttpy)')

t = tpc()
Z = Z.round(1E-2)
t = tpc() - t
e = np.linalg.norm(X - Z.full())/np.linalg.norm(X)

print(f'Ranks          : {Z.r[1:-1]}')
print(f'Error          : {e:8.2e}')
print(f'Time (sec)     : {t:-8.4f}')

---------------- Construction (ttpy)
Ranks          : [4 8 5 1]
Error          : 6.24e-07
Time (sec)     :   0.0041
---------------- Rounding (ttpy)
Ranks          : [3 4 3 1]
Error          : 1.42e-03
Time (sec)     :   0.0006


In [9]:
print('---------------- Construction (ttpy)')

t = tpc()
Z = tt.tensor(X, 1E-6)
Z = Z * Z
Z = Z.round(1E-4)
t = tpc() - t
e = np.linalg.norm(X*X - Z.full())/np.linalg.norm(X*X)

print(f'Ranks          : {Z.r[1:-1]}')
print(f'Error          : {e:8.2e}')
print(f'Time (sec)     : {t:-8.4f}')

---------------- Construction (ttpy)
Ranks          : [4 6 4 1]
Error          : 2.45e-05
Time (sec)     :   0.0062


In [ ]:
print('---------------- Construction (ttpy)')

t = tpc()
Z = tt.tensor(X, 1E-6)
Z = Z * Z
t = tpc() - t
e = np.linalg.norm(X*X - Z.full())/np.linalg.norm(X*X)

print(f'Ranks          : {Z.r[1:-1]}')
print(f'Error          : {e:8.2e}')
print(f'Time (sec)     : {t:-8.4f}')

In [47]:
print('---------------- Construction (ttpy)')

X = X.swapaxes(0, 1)
t = tpc()
Z = tt.tensor(X, 1E-6)
t = tpc() - t
e = np.linalg.norm(X - Z.full())/np.linalg.norm(X)

print(f'Ranks          : {Z.r[1:-1]}')
print(f'Error          : {e:8.2e}')
print(f'Time (sec)     : {t:-8.4f}')

print('---------------- Rounding (ttpy)')

t = tpc()
Z = Z.round(1E-4)
t = tpc() - t
e = np.linalg.norm(X - Z.full())/np.linalg.norm(X)

print(f'Ranks          : {Z.r[1:-1]}')
print(f'Error          : {e:8.2e}')
print(f'Time (sec)     : {t:-8.4f}')

---------------- Construction (ttpy)
Ranks          : [6 6 7 4]
Error          : 3.32e-08
Time (sec)     :   0.0044
---------------- Rounding (ttpy)
Ranks          : [6 6 5 4]
Error          : 9.17e-06
Time (sec)     :   0.0006


<div class="end"></div>